In [1]:
pip install -q -U google-generativeai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.16.2 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import google.generativeai as genai
import random
import re
import time
import pickle

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ["GEMINI_API_KEY"] = 'YOUR GEMINI API KEY'

In [6]:
# Generated by AI, will be used to add variety mountain names in dataset
mountains = [
    "Everest", "K2", "Kangchenjunga", "Lhotse", "Makalu", "Cho Oyu",
    "Dhaulagiri I", "Manaslu", "Nanga Parbat", "Annapurna I", "Gasherbrum I", "Broad Peak",
    "Gasherbrum II", "Shishapangma", "Mont Blanc", "Matterhorn", "Kilimanjaro", "Fuji",
    "Denali", "Aconcagua", "Elbrus", "Vinson Massif", "Puncak Jaya", "Kenya",
    "Rainier", "Shasta", "Hood", "Pikes Peak", "Mitchell", "Washington",
    "Longs Peak", "Whitney", "Saint Helens", "Baker", "Olympus (Greece)", "Etna",
    "Vesuvius", "Teide", "Ben Nevis", "Scafell Pike", "Snowdon", "Ventoux",
    "Zugspitze", "Grossglockner", "Dufourspitze", "Gran Paradiso", "Cook", "Aspiring",
    "Ngauruhoe", "Ruapehu", "Taranaki", "Kirkjufell", "Hvannadalshnúkur", "Snaefellsjokull",
    "Kebnekaise", "Galdhopiggen", "Stetind", "Logan", "Saint Elias", "Robson",
    "Columbia", "Fairweather", "Assiniboine", "Cerro Torre", "Fitz Roy", "Roraima",
    "Chimborazo", "Cotopaxi", "Huáscarán", "Yerupajá", "Sajama", "Illimani",
    "Illampu", "Licancabur", "Ojos del Salado", "Llullaillaco", "Ararat", "Damavand",
    "Meru", "Cameroon", "Toubkal", "Ras Dashen", "Stanley", "Speke",
    "Baker (Africa)", "Table Mountain", "Drakensberg", "Kosciuszko", "Townsend", "Bogong",
    "Feathertop", "Ossa", "Wellington", "Uluru (Ayers Rock)", "Warning", "Barney",
    "Bartle Frere", "Sonder", "Zeil", "Kinabalu", "Agung", "Bromo",
    "Rinjani", "Semeru", "Kerinci", "Fansipan", "Pulag", "Apo",
    "Hallasan", "Iwate", "Ontake", "Asama", "Kita", "Hotaka",
    "Yari", "Tate", "Shirane", "Bandai", "Ibuki", "Daisen",
    "Tsurugi", "Unzen", "Aso", "Sakurajima", "Asahidake", "Tokachi",
    "Rishiri", "Shiretoko", "Kamuydake", "Tarumae", "Yotei", "Zao",
    "Chokai", "Gassan", "Adatara", "Nantai", "Akagi", "Tsukuba",
    "Tanigawa", "Aizukoma", "Hiuchigatake", "Chausudake", "Nasu", "Norikura",
    "Haku", "Yakedake", "Karamatsudake", "Tsubakurodake", "Jonen", "Okuhotakadake",
    "Maehotakadake", "Nishihotakadake", "Kasagatake", "Washibadake", "Suisho", "Yakushi",
    "Kurodake", "Asahi (Hokkaido)", "Tomuraushi", "Pippu", "Rausu", "Iō",
    "Kaimondake", "Kirishima", "Sobo", "Aso (Kumamoto)", "Yufu", "Etna",
    "Blanc", "Fuji", "Rainier", "Shasta", "Hood", "Whitney",
    "Logan", "Elbrus", "Ararat", "Olympus", "Kilimanjaro", "Cook"
]
print(len(mountains))

180


In [7]:
# Text formats for generation
text_formats = ['whatsup conversation', 'news', 'book', 'adverstion', 'academic paper', 'tutorial', 'podcast transcript', 'blog']
# Text themes for generation
text_themes =  ['mountains', 'rivers', 'journeys', 'mountain regions (cities and countries)', 'culture of peoples living near mountainous areas', 'national park']
# Text sizes for generation
text_size = ['small', 'medium'] 

In [8]:
# Configure model
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

# Load model. i used "gemini-2.0-flash-exp"
model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
)

In [11]:
def generate_text(text_formats, text_themes, text_size, mountains):
    """
    Generating text, using Gemini with random parameters. In half cases lower text,
    and change half of the mountain names to one from the list mountains.

    Args:
    text_formats (list): possible text formats
    text_themes (list): possible text themes
    text_size (list): possible text sizes
    mountains (list): possible mountain names

    Returns:
    dictionary {'text': *str, generated and edited text*,
                'text_format': *str, selected format from text_formats*,
                'text_theme': *str, selected theme from text_themes*,
                'is_lower': *bool, or lowercase text*}
    """

    # Random params for text
    rand_format = random.choice(text_formats)
    rand_size = random.choice(text_size)
    rand_theme = random.choice(text_themes)
    
    response = {'text': '', 'text_format': rand_format, 'text_theme': rand_theme, 'text_size': rand_size, 'is_lower': False}

    # start chat with model
    chat_session = model.start_chat(
      history=[
        {
          "role": "model",
          "parts": [
            "I will strictly and professionally perform the work provided to me by the user, without deviating from the given instructions."
          ],
        },
      ]
    )
    # prompt, params inserts here.
    prompt = f"""
I will use you to generate a dataset for named entity recognition (NER). Your task is to create a human-like text that adheres to the following rules and guidelines:  

### Task Requirements:
1. **Content Focus**:  
   - Include common names of geographical objects such as rivers, lakes, oceans, cities, forests, etc., with an emphasis on mountain names and mountain chains.  

2. **Square Bracket Usage**:  
   - **Mountain Names**: Always enclose the name of a mountain or mountain chain in square brackets, e.g., [Everest], [Himalayas].  
   - **Other Names**: Do NOT enclose names of other objects (e.g., rivers, cities, lakes) in square brackets. This rule is critical and any deviation will be penalized.  

3. **Examples**:  
   - Correct: "[Everest] is the highest mountain, Amazon is the biggest river. Amazon River is located in Brazil and [Everest] is located on the border between Nepal and China."  
   - Incorrect: "[Amazon River] is located in Brazil and Everest is located on the border between Nepal and China."  

### Text Output Guidelines:
1. The generated text must not include any phrases like "Okay, here goes:" or similar unnecessary additions.  
2. Do not write headlines or use emojis in the text.  
3. After generating the text, double-check for compliance with the square bracket rules outlined above.  

### Text Parameters:
- **Text Format**: {rand_format}  
- **Text Theme**: {rand_theme}  
- **Text Size**: {rand_size}  

### Post-Processing:
After generating the text:
1. Verify compliance with the square bracket rules.
2. Provide the generated text with clear adherence to the requirements.

Failure to follow these rules will result in the text being marked as invalid."""

    # Recieve models answer
    generated_text = chat_session.send_message(prompt).text
    
    # Changing mountain names in half cases for the variety. It will be good for model to not just remember some names, but understand context.
    matches = re.finditer(r'\[(.*?)\]', generated_text)
    for match in matches:
        original_match_with_brackets = match.group(0)
        if random.random() < 0.5:
            replacement_mountain = random.choice(mountains)
            generated_text = generated_text.replace(original_match_with_brackets, f"[{replacement_mountain}]", 1)
            
    
    # High literals is hint for the model. Lowercase in half cases.
    if random.random() < 0.5:
        generated_text = generated_text.lower()
        response['is_lower'] = True
            
    response['text'] = generated_text
    
    return(response)

def generate_dataset(text_formats, text_themes, text_size, mountains, n):
    """
    Generate n responses from Gemini.
    
    Args:
    text_formats (list): possible text formats
    text_themes (list): possible text themes
    text_size (list): possible text sizes
    mountains (list): possible mountain names
    n (int): number of requests

    Returns:
    list of dictionaries 
    {'text': *str, generated and edited text*,
    'text_format': *str, selected format from text_formats*,
    'text_theme': *str, selected theme from text_themes*,
    'is_lower': *bool, or lowercase text*}
    """
    
    data = []
    start_time = 0

    # Define delay and retry count for requests.
    delay = 0.1
    retry_count = 9

    # Generating cycle
    for i in range(n):
        
        # Chek if quota exceeded (10 requests per minute)
        if i % 10 == 0:
            if start_time !=0:
                current_time = time.time()
                time_since_start = current_time - start_time
                if time_since_start < 65:
                    time_to_sleep = 65 - time_since_start
                    print(f"Delay {time_to_sleep:.2f} seconds to send next 10 requests")
                    time.sleep(time_to_sleep)
                    
            start_time = time.time()
            
        try:
            response = generate_text(text_formats, text_themes, text_size, mountains)
            data.append(response)
            print(f'{i+1}) Theme: {response["text_theme"]}. Format: {response["text_size"]} {response["text_format"]}')
            
        # Sometimes it returns 429 error even if quota not exceeded. But in this case its not a problem to just ignore then find out how Google counts requests
        except Exception as e:
            if '429' in str(e):
                print(f"Received 429 error, increasing delay and retrying...")
                if retry_count > 0:
                    delay *= 2 #increasing delay in case if it still not enough
                    retry_count -= 1
                else:
                    return data
                time.sleep(delay)
            else:
                print(f"Error occurred: {e}")
                return data
    return data

In [13]:
# testing single out
response = generate_text(text_formats, text_themes, text_size, mountains)
response

{'text': "The trek up [Galdhopiggen] was arduous, but the view from the top was worth it. The [Andes] mountains stretch across South America, with many of their peaks reaching impressive heights. I've always dreamed of climbing [Washington], but I know that the ascent is incredibly dangerous. The [Zeil] are beautiful in both the winter and summer. We hiked through a forest near the base of [Roraima] and saw a beautiful stream. The city of Tokyo is visible from certain points on [Mount Fuji].\n",
 'text_format': 'tutorial',
 'text_theme': 'mountains',
 'text_size': 'small',
 'is_lower': False}

In [14]:
dataset = generate_dataset(text_formats, text_themes, text_size, mountains, 300)

1) Theme: mountains. Format: small whatsup conversation
2) Theme: culture of peoples living near mountainous areas. Format: small podcast transcript
3) Theme: rivers. Format: small blog
4) Theme: national park. Format: small podcast transcript
5) Theme: journeys. Format: medium adverstion
6) Theme: national park. Format: small book
7) Theme: rivers. Format: medium podcast transcript
8) Theme: rivers. Format: medium tutorial
9) Theme: mountain regions (cities and countries). Format: small tutorial
10) Theme: culture of peoples living near mountainous areas. Format: small academic paper
Delay 45.95 seconds to send next 10 requests
11) Theme: journeys. Format: small podcast transcript
12) Theme: mountains. Format: small adverstion
13) Theme: culture of peoples living near mountainous areas. Format: medium whatsup conversation
14) Theme: rivers. Format: small news
15) Theme: journeys. Format: small tutorial
16) Theme: culture of peoples living near mountainous areas. Format: medium tutoria

In [15]:
dataset[:5]

[{'text': "Hey, did you know that [Chokai] is actually a volcano? It's so cool! I've always wanted to visit [Tsukuba], maybe we should plan a trip? My friend went hiking in [the Andes] last year and said it was incredible. Have you ever seen a photo of [Hood]? It looks so dangerous. Oh, and I read that the Nile River is the longest in the world.\n",
  'text_format': 'whatsup conversation',
  'text_theme': 'mountains',
  'text_size': 'small',
  'is_lower': False},
 {'text': 'The Sherpa people, who inhabit the high-altitude regions of Nepal near [Aconcagua], have a rich cultural heritage deeply intertwined with the [Broad Peak]. Their lives are shaped by the challenging terrain and the spiritual significance they attribute to the mountains. They believe that [Kangchenjunga] is home to a powerful deity. The sacred rivers, such as Koshi, flow from the melting glaciers of the [Kaimondake] range, providing life-sustaining water for communities downstream. These communities rely on the local 

In [16]:
with open('mountain_ner_data.pkl', 'wb') as file:
    pickle.dump(dataset, file)